In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# from PIL import Image
# import os, sys
# import glob


# IMAGE_EXTENSTION = '.jpg'
# path = '/media/dean/datastore/datasets/kache_ai/frames_dev/'
# imgs = glob.glob(os.path.join(path,'**', '*'+IMAGE_EXTENSTION))

# def resize():
#     for img in imgs:
#         if os.path.isfile(img):
#             im = Image.open(img)
#             f, e = os.path.splitext(img)
#             imResize = im.resize((1280,1080), Image.ANTIALIAS)
#             imResize.save(f + '-resized.jpg', 'JPEG', quality=99)
#             print('resized:', img)
#             !rm -f $img
            
# resize()

In [3]:
import os
import glob
import shutil
import hashlib
from collections import OrderedDict
import pandas as pd
import time
import datetime
import pickle
from utils import datasets
from enum import Enum
import pynmea2

In [4]:
IMAGE_EXTENSTION = '.jpg'

In [5]:

class Format(Enum):
    atm = 0
    ros = 1
    gopro = 2
    raw = 3

In [6]:
# Try on small subset ..
input_format = Format.raw

## --- Raw Image Data Handler --- ##
if input_format == Format.raw:
    frames_dir = '/media/dean/datastore/datasets/kache_ai/frames_dev/'
    save_dir = os.path.join(frames_dir,'kache_set/')
    os.makedirs(save_dir, 0o755 , exist_ok = True )

    img_data = []
    lookup_table = OrderedDict()
    img_dict = OrderedDict()
    bag_path = os.path.join(frames_dir)

    img_dict['bag_path'] = frames_dir
    img_dict['frames'] = []
    img_dict['csv_path'] = ''

    # Get all images
    imgs = glob.glob(os.path.join(bag_path,'**', '*'+IMAGE_EXTENSTION))
    print(len(imgs))
    for idx, img in enumerate(sorted(set(imgs))):
        print(img)
        d = OrderedDict()
        d['frame_path'] = img
        d['frame_idx'] = int(idx)
        hex_val = os.path.abspath(d['frame_path']).replace('/', '^^')+'---'+os.path.abspath(img_dict['csv_path']).replace('/', '^^')
        hash_object = hashlib.sha1(str.encode(hex_val))
        hex_dig = hash_object.hexdigest()
        shutil.copyfile(d['frame_path'], os.path.join(save_dir, str(hex_dig)+IMAGE_EXTENSTION))
        d['dataset_path'] = os.path.join(save_dir, str(hex_dig)+IMAGE_EXTENSTION)
        d['hash_key'] = hex_dig
        d['hash_val'] = hex_val
        d['frame'] = str(d['dataset_path'])
        lookup_table[d['hash_key']] = d['hash_val'] 
        old_path = os.path.join(bag_path, 'frames', os.path.split(d['frame_path'])[-1])

        if d.get('frame', None): img_dict['frames'].append(d)
    if img_dict['frames']: img_data.append((bag_path,img_dict))


## --- ROS Bag Handler --- ##
elif input_format == Format.ros:
    HEADER_ROW=['bag', 'time_sec', 'time_nsec', 'GPS', 'v_ego', 'key_event', 'frame_name']
    frames_dir = '/media/dean/datastore/datasets/kache_ai/frames_dev/'
    save_dir = os.path.join(frames_dir,'kache_set/')
    os.makedirs(save_dir, 0o755 , exist_ok = True )

    img_data = []
    lookup_table = OrderedDict()
    img_dict = OrderedDict()
    bag_path = os.path.join(frames_dir)
    img_dict['bag_path'] = frames_dir
    img_dict['frames'] = []
    img_dict['csv_path'] = os.path.join(bag_path, 'frames.csv')
    csv_logger = pd.read_csv(img_dict['csv_path'], names=HEADER_ROW, skiprows=1)

    # Get all images
    imgs = glob.glob(os.path.join(bag_path,'**/', '*.JPG'))

    for idx, img in enumerate(sorted(set(imgs))):
        d = OrderedDict()
        d['frame_path'] = img
        d['frame_idx'] = int(idx)
        hex_val = os.path.abspath(d['frame_path']).replace('/', '^^')+'---'+os.path.abspath(img_dict['csv_path']).replace('/', '^^')
        hash_object = hashlib.sha1(str.encode(hex_val))
        hex_dig = hash_object.hexdigest()
        shutil.copyfile(d['frame_path'], os.path.join(save_dir, str(hex_dig)+'.jpg'))
        d['dataset_path'] = os.path.join(save_dir, str(hex_dig)+'.jpg')
        d['hash_key'] = hex_dig
        d['hash_val'] = hex_val
        lookup_table[d['hash_key']] = d['hash_val'] 
        old_path = os.path.join(bag_path, 'frames', os.path.split(d['frame_path'])[-1])

        for log in [x for x in csv_logger.as_matrix() if os.path.join(bag_path, 'frames', x[6].rstrip('/')) == old_path.rstrip('/')]:
            d['bag_name'] = log[0]
            d['time_sec'] = log[1]
            d['time_readable'] = time.ctime(int(log[1]))

            d['time_nsec'] = log[2]
            d['GPS'] = log[3]
            d['v-ego'] = log[4]

            d['key_event'] = log[5]
            d['frame'] = log[6]

        if d.get('frame', None): img_dict['frames'].append(d)
    if img_dict['frames']: img_data.append((bag_path,img_dict))


        
        
        
        
        
pickle_file = os.path.join(save_dir, 'kacheai_logs.pickle')
pickle_dict = {'img_data':img_data,'lookup_table':lookup_table}
print('Saving to Pickle File:', pickle_file)
with open(pickle_file,"wb") as pickle_out:
    pickle.dump(pickle_dict, pickle_out)



2608
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frame-1-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frame-10-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frame-100-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frame-101-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frame-102-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frame-103-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frame-104-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frame-105-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frame-106-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frame-107-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frame-108-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frame-109-resized.jpg
/media/dean/datastore/datasets/kache_a

/media/dean/datastore/datasets/kache_ai/frames_dev/frames10/frame-180-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames10/frame-181-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames10/frame-182-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames10/frame-183-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames10/frame-184-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames10/frame-185-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames10/frame-186-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames10/frame-187-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames10/frame-188-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames10/frame-189-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames10/frame-19-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames10/frame-190-resized.jpg
/media/dean/datas

/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-127-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-128-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-129-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-13-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-130-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-131-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-132-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-133-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-134-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-135-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-136-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-137-resized.jpg
/media/dean/datastore/dataset

/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-49-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-490-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-491-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-492-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-493-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-494-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-495-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-496-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-497-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-498-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-499-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-5-resized.jpg
/media/dean/datastore/datasets/

/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-93-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-930-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-931-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-932-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-933-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-934-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-935-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-936-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-937-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-938-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-939-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames3/frame-94-resized.jpg
/media/dean/datastore/datasets

/media/dean/datastore/datasets/kache_ai/frames_dev/frames7/frame-26-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames7/frame-27-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames7/frame-28-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames7/frame-29-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames7/frame-3-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames7/frame-30-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames7/frame-31-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames7/frame-32-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames7/frame-33-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames7/frame-34-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames7/frame-35-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames7/frame-36-resized.jpg
/media/dean/datastore/datasets/kache_ai/f

/media/dean/datastore/datasets/kache_ai/frames_dev/frames9/frame-156-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames9/frame-157-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames9/frame-158-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames9/frame-159-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames9/frame-16-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames9/frame-160-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames9/frame-161-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames9/frame-162-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames9/frame-163-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames9/frame-164-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames9/frame-165-resized.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames9/frame-166-resized.jpg
/media/dean/datastore/dataset

In [7]:
print(img_data)

[('/media/dean/datastore/datasets/kache_ai/frames_dev/', OrderedDict([('bag_path', '/media/dean/datastore/datasets/kache_ai/frames_dev/'), ('frames', [OrderedDict([('frame_path', '/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frame-1-resized.jpg'), ('frame_idx', 0), ('dataset_path', '/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/2fc93678f11841e6ebcd12e4507b1c91e0480de9.jpg'), ('hash_key', '2fc93678f11841e6ebcd12e4507b1c91e0480de9'), ('hash_val', '^^media^^dean^^datastore^^datasets^^kache_ai^^frames_dev^^frames^^frame-1-resized.jpg---^^media^^dean^^datastore^^datasets^^Scripts'), ('frame', '/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/2fc93678f11841e6ebcd12e4507b1c91e0480de9.jpg')]), OrderedDict([('frame_path', '/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frame-10-resized.jpg'), ('frame_idx', 1), ('dataset_path', '/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/b60d4f1e4a90b3f6fbfef9ec402b10db9b2ce809.jpg'), ('hash_ke

In [8]:
WORKING_DIR = '/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/'

!aws s3 sync $WORKING_DIR s3://kache-scalabel/kache_ai/frames/

upload: ../kache_ai/frames_dev/kache_set/00241f6bb885bff7fe3b95ed4d3599b5d8b6927d.jpg to s3://kache-scalabel/kache_ai/frames/00241f6bb885bff7fe3b95ed4d3599b5d8b6927d.jpg
upload: ../kache_ai/frames_dev/kache_set/00c02c6c5025be5e3094853dc8d5549987ed1c85.jpg to s3://kache-scalabel/kache_ai/frames/00c02c6c5025be5e3094853dc8d5549987ed1c85.jpg
upload: ../kache_ai/frames_dev/kache_set/0031bd1a26acb3f5f5af824644746430ca24b4d7.jpg to s3://kache-scalabel/kache_ai/frames/0031bd1a26acb3f5f5af824644746430ca24b4d7.jpg
upload: ../kache_ai/frames_dev/kache_set/00b0621f77a200d0e1a38034464168d2b7fe14c3.jpg to s3://kache-scalabel/kache_ai/frames/00b0621f77a200d0e1a38034464168d2b7fe14c3.jpg
upload: ../kache_ai/frames_dev/kache_set/0052216804b28f1a0354fc19db77b50f4bc62379.jpg to s3://kache-scalabel/kache_ai/frames/0052216804b28f1a0354fc19db77b50f4bc62379.jpg
upload: ../kache_ai/frames_dev/kache_set/009c694e2e5a55d93ce865386befcb508f32c704.jpg to s3://kache-scalabel/kache_ai/frames/009c694e2e5a55d93ce865386

upload: ../kache_ai/frames_dev/kache_set/04c0885dfb6591cda82baed6d4ca7e6eaa4a93f1.jpg to s3://kache-scalabel/kache_ai/frames/04c0885dfb6591cda82baed6d4ca7e6eaa4a93f1.jpg
upload: ../kache_ai/frames_dev/kache_set/04c6a3ebfe80599cb0a20e5f0b13a8a1de867d4a.jpg to s3://kache-scalabel/kache_ai/frames/04c6a3ebfe80599cb0a20e5f0b13a8a1de867d4a.jpg
upload: ../kache_ai/frames_dev/kache_set/04b88e408e1715ed5a41319cea5b16e87baf8992.jpg to s3://kache-scalabel/kache_ai/frames/04b88e408e1715ed5a41319cea5b16e87baf8992.jpg
upload: ../kache_ai/frames_dev/kache_set/05178cd177605d9d305cbd2f23e707414527898e.jpg to s3://kache-scalabel/kache_ai/frames/05178cd177605d9d305cbd2f23e707414527898e.jpg
upload: ../kache_ai/frames_dev/kache_set/05105944abd87a7e552614ad1c03b7910d9c5330.jpg to s3://kache-scalabel/kache_ai/frames/05105944abd87a7e552614ad1c03b7910d9c5330.jpg
upload: ../kache_ai/frames_dev/kache_set/05114d509a203e624457b91934aba23dfabecad9.jpg to s3://kache-scalabel/kache_ai/frames/05114d509a203e624457b9193

upload: ../kache_ai/frames_dev/kache_set/0939dfb99c4837a80bdb88047d967392b65b469d.jpg to s3://kache-scalabel/kache_ai/frames/0939dfb99c4837a80bdb88047d967392b65b469d.jpg
upload: ../kache_ai/frames_dev/kache_set/094dff93f2e85fa0acf5a53b3ef1ec6bd650477e.jpg to s3://kache-scalabel/kache_ai/frames/094dff93f2e85fa0acf5a53b3ef1ec6bd650477e.jpg
upload: ../kache_ai/frames_dev/kache_set/095f04e2730951cd1486337592505e95a432ea35.jpg to s3://kache-scalabel/kache_ai/frames/095f04e2730951cd1486337592505e95a432ea35.jpg
upload: ../kache_ai/frames_dev/kache_set/09213fe0b1f9e734ca401ce4f9c14575bcbadce1.jpg to s3://kache-scalabel/kache_ai/frames/09213fe0b1f9e734ca401ce4f9c14575bcbadce1.jpg
upload: ../kache_ai/frames_dev/kache_set/099bf89840fa3a388363ec7d064606d8f04cb531.jpg to s3://kache-scalabel/kache_ai/frames/099bf89840fa3a388363ec7d064606d8f04cb531.jpg
upload: ../kache_ai/frames_dev/kache_set/09c16a23ed4ab2c47d734c39abce2d200d4a9ea2.jpg to s3://kache-scalabel/kache_ai/frames/09c16a23ed4ab2c47d734c39a

upload: ../kache_ai/frames_dev/kache_set/0f6c9b400594553e13db388afdb06984e858df8a.jpg to s3://kache-scalabel/kache_ai/frames/0f6c9b400594553e13db388afdb06984e858df8a.jpg
upload: ../kache_ai/frames_dev/kache_set/0e04785949ffd05e6b0b42f02a9f956902e9e9eb.jpg to s3://kache-scalabel/kache_ai/frames/0e04785949ffd05e6b0b42f02a9f956902e9e9eb.jpg
upload: ../kache_ai/frames_dev/kache_set/0f96da08f638ef170a90b8e0f955a27ec1be0b0a.jpg to s3://kache-scalabel/kache_ai/frames/0f96da08f638ef170a90b8e0f955a27ec1be0b0a.jpg
upload: ../kache_ai/frames_dev/kache_set/0f736af6eff00348e2e4e373f88a9f9a7dd42be1.jpg to s3://kache-scalabel/kache_ai/frames/0f736af6eff00348e2e4e373f88a9f9a7dd42be1.jpg
upload: ../kache_ai/frames_dev/kache_set/0e73eda76bba13f4f30439735ef795a34bfc1bd8.jpg to s3://kache-scalabel/kache_ai/frames/0e73eda76bba13f4f30439735ef795a34bfc1bd8.jpg
upload: ../kache_ai/frames_dev/kache_set/0f6ede7a42277dd45d0c9069f62039daa0611dd2.jpg to s3://kache-scalabel/kache_ai/frames/0f6ede7a42277dd45d0c9069f

upload: ../kache_ai/frames_dev/kache_set/13d9bf35175eaaba05d1948b43dbc849be012c8a.jpg to s3://kache-scalabel/kache_ai/frames/13d9bf35175eaaba05d1948b43dbc849be012c8a.jpg
upload: ../kache_ai/frames_dev/kache_set/13c8e85180948cd48cdd2abcf3028ee21a036be5.jpg to s3://kache-scalabel/kache_ai/frames/13c8e85180948cd48cdd2abcf3028ee21a036be5.jpg
upload: ../kache_ai/frames_dev/kache_set/13d984317b8117a5d88d2a427bb63639c3a3d827.jpg to s3://kache-scalabel/kache_ai/frames/13d984317b8117a5d88d2a427bb63639c3a3d827.jpg
upload: ../kache_ai/frames_dev/kache_set/139da7d2734d8cc84db5bfb651acd2c5c79bd9a0.jpg to s3://kache-scalabel/kache_ai/frames/139da7d2734d8cc84db5bfb651acd2c5c79bd9a0.jpg
upload: ../kache_ai/frames_dev/kache_set/143300c7b3089e5c175fcf7d36afcea3e45d3c65.jpg to s3://kache-scalabel/kache_ai/frames/143300c7b3089e5c175fcf7d36afcea3e45d3c65.jpg
upload: ../kache_ai/frames_dev/kache_set/13e19e267aed97c60935cec3cca5b32215707a5f.jpg to s3://kache-scalabel/kache_ai/frames/13e19e267aed97c60935cec3c

...

upload: ../kache_ai/frames_dev/kache_set/ec9df3f3222c396a9c3fd3b9b41314c75a13212e.jpg to s3://kache-scalabel/kache_ai/frames/ec9df3f3222c396a9c3fd3b9b41314c75a13212e.jpg
upload: ../kache_ai/frames_dev/kache_set/ecf4507d0a6c5aa16e69fee8c4d4ee566174b836.jpg to s3://kache-scalabel/kache_ai/frames/ecf4507d0a6c5aa16e69fee8c4d4ee566174b836.jpg
upload: ../kache_ai/frames_dev/kache_set/ecc0a3aa767831c0fa4bba80a3acbbfb89b8c970.jpg to s3://kache-scalabel/kache_ai/frames/ecc0a3aa767831c0fa4bba80a3acbbfb89b8c970.jpg
upload: ../kache_ai/frames_dev/kache_set/ecc0b20e4004cd15099cd6b520f66761269e644c.jpg to s3://kache-scalabel/kache_ai/frames/ecc0b20e4004cd15099cd6b520f66761269e644c.jpg
upload: ../kache_ai/frames_dev/kache_set/ecccf6a6fe5854a2cb025dbe74d7fd1f2faf9d19.jpg to s3://kache-scalabel/kache_ai/frames/ecccf6a6fe5854a2cb025dbe74d7fd1f2faf9d19.jpg
upload: ../kache_ai/frames_dev/kache_set/ec7823f9de49a3a8d7c039e910057d2c1f0f8833.jpg to s3://kache-scalabel/kache_ai/frames/ec7823f9de49a3a8d7c039e91

upload: ../kache_ai/frames_dev/kache_set/f08c6c7ab88ad3b8744c5a62f3c72adcc7bd5c1e.jpg to s3://kache-scalabel/kache_ai/frames/f08c6c7ab88ad3b8744c5a62f3c72adcc7bd5c1e.jpg
upload: ../kache_ai/frames_dev/kache_set/f13612e62c3b45a48671b88a85d414b6706f7b2a.jpg to s3://kache-scalabel/kache_ai/frames/f13612e62c3b45a48671b88a85d414b6706f7b2a.jpg
upload: ../kache_ai/frames_dev/kache_set/f13087efba66e2a1760221cc21f441966d11144b.jpg to s3://kache-scalabel/kache_ai/frames/f13087efba66e2a1760221cc21f441966d11144b.jpg
upload: ../kache_ai/frames_dev/kache_set/f17cbeff19c9f0acaf4969c065d1bb608875b2e5.jpg to s3://kache-scalabel/kache_ai/frames/f17cbeff19c9f0acaf4969c065d1bb608875b2e5.jpg
upload: ../kache_ai/frames_dev/kache_set/f20a6a539aed48eecb45d7fb50f623c353802482.jpg to s3://kache-scalabel/kache_ai/frames/f20a6a539aed48eecb45d7fb50f623c353802482.jpg
upload: ../kache_ai/frames_dev/kache_set/f1c0857bca7f8d1d0580b79e1c201cadf631e4ec.jpg to s3://kache-scalabel/kache_ai/frames/f1c0857bca7f8d1d0580b79e1

upload: ../kache_ai/frames_dev/kache_set/f68303da0df405679ec100247541a3fb9ebabd18.jpg to s3://kache-scalabel/kache_ai/frames/f68303da0df405679ec100247541a3fb9ebabd18.jpg
upload: ../kache_ai/frames_dev/kache_set/f61c03f16b46584ac6f45764742043afa401060d.jpg to s3://kache-scalabel/kache_ai/frames/f61c03f16b46584ac6f45764742043afa401060d.jpg
upload: ../kache_ai/frames_dev/kache_set/f62322b562d0693f9cf5f7d0c3ed931b1ebec28a.jpg to s3://kache-scalabel/kache_ai/frames/f62322b562d0693f9cf5f7d0c3ed931b1ebec28a.jpg
upload: ../kache_ai/frames_dev/kache_set/f6128f851319a564bf7af51acd1272de52199dbf.jpg to s3://kache-scalabel/kache_ai/frames/f6128f851319a564bf7af51acd1272de52199dbf.jpg
upload: ../kache_ai/frames_dev/kache_set/f6c7ccbf920d7bd3020bbe73c315a8e07a854309.jpg to s3://kache-scalabel/kache_ai/frames/f6c7ccbf920d7bd3020bbe73c315a8e07a854309.jpg
upload: ../kache_ai/frames_dev/kache_set/f5d32e62f4053d75231d7626c49f5d06be220d7a.jpg to s3://kache-scalabel/kache_ai/frames/f5d32e62f4053d75231d7626c

upload: ../kache_ai/frames_dev/kache_set/fb6872e756255e5d54991068e477512474dc04be.jpg to s3://kache-scalabel/kache_ai/frames/fb6872e756255e5d54991068e477512474dc04be.jpg
upload: ../kache_ai/frames_dev/kache_set/fabe1a7b5312aa632f561ac036b1b35dde44a36b.jpg to s3://kache-scalabel/kache_ai/frames/fabe1a7b5312aa632f561ac036b1b35dde44a36b.jpg
upload: ../kache_ai/frames_dev/kache_set/fb5fa8b274d871339b173f7c0dda625162ac8b30.jpg to s3://kache-scalabel/kache_ai/frames/fb5fa8b274d871339b173f7c0dda625162ac8b30.jpg
upload: ../kache_ai/frames_dev/kache_set/fbbc203e8eb439442434d88669ecc66b60abbcc7.jpg to s3://kache-scalabel/kache_ai/frames/fbbc203e8eb439442434d88669ecc66b60abbcc7.jpg
upload: ../kache_ai/frames_dev/kache_set/fbef5de2cfdbfed90b7d72e81a3b112d3f8c73ec.jpg to s3://kache-scalabel/kache_ai/frames/fbef5de2cfdbfed90b7d72e81a3b112d3f8c73ec.jpg
upload: ../kache_ai/frames_dev/kache_set/fb875432a91440917b30e32d7307e7260b8b7db3.jpg to s3://kache-scalabel/kache_ai/frames/fb875432a91440917b30e32d7

upload: ../kache_ai/frames_dev/kache_set/ff766af5b2c049a751bd52b3bcccff9f2694d88c.jpg to s3://kache-scalabel/kache_ai/frames/ff766af5b2c049a751bd52b3bcccff9f2694d88c.jpg
upload: ../kache_ai/frames_dev/kache_set/ff20db947527d9473e387c2971cbd4fbbe76262b.jpg to s3://kache-scalabel/kache_ai/frames/ff20db947527d9473e387c2971cbd4fbbe76262b.jpg
upload: ../kache_ai/frames_dev/kache_set/ff386f5c222dfabf90b094a6a734a63366b06e44.jpg to s3://kache-scalabel/kache_ai/frames/ff386f5c222dfabf90b094a6a734a63366b06e44.jpg
upload: ../kache_ai/frames_dev/kache_set/ff38d926aa5467b6a5159db5fda731cd9b07eb7a.jpg to s3://kache-scalabel/kache_ai/frames/ff38d926aa5467b6a5159db5fda731cd9b07eb7a.jpg
upload: ../kache_ai/frames_dev/kache_set/feb7914e38ff0227361185368a35009dcc8492b1.jpg to s3://kache-scalabel/kache_ai/frames/feb7914e38ff0227361185368a35009dcc8492b1.jpg
upload: ../kache_ai/frames_dev/kache_set/ff126d75358c884ace866ba51f6ec9e80bf245e8.jpg to s3://kache-scalabel/kache_ai/frames/ff126d75358c884ace866ba51

In [15]:


WORKING_DIR = '/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/'
kache_set = datasets.DataFormatter(annotations_list = os.path.join(WORKING_DIR,'kacheai_logs.pickle'), image_list = WORKING_DIR, input_format = datasets.Format.kache,
                                 output_path = os.path.join(WORKING_DIR,'scalabel_files/'),
                                 trainer_prefix = 'COCO_train2014_0000', 
                                 s3_bucket = 'kache-scalabel/kache_ai/frames/', check_s3 = False, use_static_categories = True)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



DATA_CONFIG: /media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/scalabel_files/inference/cfg/bdd100k.data
MODEL_CONFIG: /media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/scalabel_files/inference/cfg/yolov3-bdd100k.cfg
MODEL_WEIGHTS: /media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/scalabel_files/inference/models/yolov3-bdd100k_4300.weights
Initializing Annotation Pipeline via Darknet ... \o/ \o/
/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/3183ce026b44a2ae72551dd5a33324e5e9c6bf2c.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/b7bd674e7d01f4540018c21dfb612fd5e212573d.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/de9f019819c2376ebe5d3957c4a45512f7363168.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/d91fcffce9634a665bb3000514e5da5774894ee2.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/c3e6fa3a973b6c865cdf9e49abd0be0e10f66ae1.jpg
/media/dean/datastore/datasets/kache_a

/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/f7a50b0de3797706173b715adbe3cf37d3b42d51.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/64f813ed96e9e57192528b292fc84ab0f75d8f90.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/e37b5def3316bb62f54e3d30de243762e588145f.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/c9b7baec9f3b64139ce566c2a50181e913ed2bcc.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/3ba84dafbba73d76c0464873b57bfa01b918cf24.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/2a087480dde2dc5a9765751859a5223e0257730a.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/a88ea0ef596984ddbfea146b0dc5bda6331ff802.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/721034aa3b1c6765d6255fafbff8f0dfa5c54730.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/ed8f7e1ede5d63975ece5a1e39859e7bcc7918c8.jpg
/media/dean/datastore/datasets/kache_ai/frames

/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/84e534528cdd12c8647208826e4f0ed1a1cc8107.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/42f4491670058dc4945ad1b911d828459922d2f9.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/7ac1cc9bbce12b2ec05d28ba74bbadba2b02757d.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/82ba58f37680077a95a2b8d85274ec7a1a84d8c6.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/291069855cff21ad046abcf5a381e3ab687b49e2.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/9a616d446785d7cc2854678239a8f950d9dd1ae1.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/ea9d568da76a1dd651224c2e7f3eef48ef78c58c.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/f94f616e4823bbbd0987de780982e01b794039e5.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/bb86d436c2e9bedb460cff9e1ceaab97a9f40257.jpg
/media/dean/datastore/datasets/kache_ai/frames

/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/2caa236b2863636db08a845f0d5fc683ea166ac0.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/06fe7c8ae3964a3ef7bf3c3eeea95d1088f98298.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/597f494541a850f851dc964c4cdb3a3e2d359905.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/dfca561f7fe0c52104f04c238adc5c698563d1f3.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/e5b79cfb7d26c8d9f4dda9e991773f14cad578d3.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/74111b7122349fb80307ce36ee6366f23d90258c.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/1113792a499c2ad185f2fa98b9cd8bdf67177eb9.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/36a9fe67f6589ead8d4cf1882ec236000062a027.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/cf256c676d34e2fe96047a61ecfa87a86f4abcf5.jpg
/media/dean/datastore/datasets/kache_ai/frames

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!

########### DATASET DISTRIBUTION: ###########

TRAFFIC SIGN-SPEED_SIGN | Annotations: 0  | Images:  0
TRAFFIC LIGHT | Annotations: 0  | Images:  0
RIDER | Annotations: 0  | Images:  0
TRAFFIC LIGHT-AMBER | Annotations: 0  | Images:  0
TRAFFIC SIGN | Annotations: 65  | Images:  58
CONSTRUCT-EQUIPMENT | Annotations: 0  | Images:  0
CONSTRUCT-SIGN | Annotations: 0  | Images:  0
TRAIN | Annotations: 0  | Images:  0
CONSTRUCT-POLE | Annotations: 0  | Images:  0
CONSTRUCT-CONE | Annotations: 0  | Images:  0
MOTOR | Annotations: 0  | Images:  0
PERSON | Annotations: 0  | Images:  0
TRUCK | Annotations: 70  | Images:  70
CONSTRUCT-BARREL | Annotations: 102  | Images:  89
BUS | Annotations: 0  | Images:  0
TRAFFIC LIGHT-RED | Annotations: 0  | Images:  0
TRAFFIC SIGN-STOP_SIGN | Annotations: 0  | Images:  0
BIKE | Annotations: 0  | Images:  0
TRAILER | Annotations: 0  | Images:  0
CAR | Annotations: 791  | Image

In [16]:
kache_set.export(format = datasets.Format.scalabel, paginate = False)

In [ ]:
# input_format = Format.ros

In [ ]:
# ## --- ROS Bag Handler --- ##
# if input_format == Format.ros:

#     HEADER_ROW=['bag', 'time_sec', 'time_nsec', 'GPS', 'v_ego', 'key_event', 'frame_name']
#     frames_dir = '/media/dean/datastore1/datasets/kache_ai/frames/'
#     bag_dirs = os.listdir(frames_dir)
#     save_dir = os.path.join(frames_dir,'kache_set/')
#     os.makedirs(save_dir, 0o755 , exist_ok = True )

#     img_data = []
#     lookup_table = {}
#     # if not os.path.exists(os.path.join())
#     for bag in bag_dirs:
#         if frames_dir != save_dir or '.csv' not in frames_dir:
#             img_dict = {}
#             bag_path = os.path.join(frames_dir, bag)
#             img_dict['bag_path'] = frames_dir
#             img_dict['frames'] = []
#             img_dict['csv_path'] = os.path.join(bag_path, 'frames.csv')
#             if os.path.exists(img_dict['csv_path']):
#                 csv_logger = pd.read_csv(img_dict['csv_path'], names=HEADER_ROW, skiprows=1)

#                 # Get all images
#                 imgs = glob.glob(os.path.join(bag_path,'**/', '*.jpg'))

#                 for img in sorted(set(imgs)):
#                     d = {}
#                     d['frame_path'] = img
#                     hex_val = os.path.abspath(d['frame_path']).replace('/', '^^')+'---'+os.path.abspath(img_dict['csv_path']).replace('/', '^^')
#                     hash_object = hashlib.sha1(str.encode(hex_val))
#                     hex_dig = hash_object.hexdigest()
#                     shutil.copyfile(d['frame_path'], os.path.join(save_dir, str(hex_dig)+'.jpg'))
#                     d['dataset_path'] = os.path.join(save_dir, str(hex_dig)+'.jpg')
#                     d['hash_key'] = hex_dig
#                     d['hash_val'] = hex_val
#                     lookup_table[d['hash_key']] = d['hash_val'] 
#                     old_path = os.path.join(bag_path, 'frames', os.path.split(d['frame_path'])[-1])

#                     for log in [x for x in csv_logger.as_matrix() if os.path.join(bag_path, 'frames', x[6].rstrip('/')) == old_path.rstrip('/')]:
#                         d['bag_name'] = log[0]
#                         d['time_sec'] = log[1]
#                         d['time_readable'] = time.ctime(int(log[1]))

#                         d['time_nsec'] = log[2]
#                         d['GPS'] = log[3]
#                         d['v-ego'] = log[4]
#                         d['latitude'] = None
#                         d['longitude'] = None
#                         if d['GPS']:
#                             msg = pynmea2.parse(d['GPS'])
#                             d['latitude'] = msg.lat
#                             d['longitude'] = msg.lon

#                         d['key_event'] = log[5]
#                         d['frame'] = log[6]

#                     if d.get('frame', None): img_dict['frames'].append(d)
#                 if img_dict['frames']: img_data.append((bag_path,img_dict))

# pickle_file = os.path.join(save_dir, 'kacheai_logs.pickle')
# pickle_dict = {'img_data':img_data,'lookup_table':lookup_table}
# print('Saving to Pickle File:', pickle_file)
# with open(pickle_file,"wb") as pickle_out:
#     pickle.dump(pickle_dict, pickle_out)



In [ ]:
# WORKING_DIR = '/media/dean/datastore1/datasets/kache_ai/frames/kache_set/'
# kache_set = datasets.DataFormatter(annotations_list = '/media/dean/datastore1/datasets/kache_ai/frames/kache_set/kacheai_logs.pickle', 
#                                    image_list = '/media/dean/datastore/datasets1/kache_ai/frames/kache_set/', 
#                                    input_format = datasets.Format.kache, output_path = os.path.join(WORKING_DIR,'scalabel_files'), use_cache = True,
#                                    trainer_prefix = 'COCO_train2014_0000', s3_bucket = 'kache-scalabel/kache_ai/frames/', check_s3 = False, use_static_categories = True)

In [ ]:
# kache_set.export(format = datasets.Format.scalabel, paginate = False)

In [ ]:
# kache_set.export(format = datasets.Format.scalabel, paginate = True)

In [ ]:
# !cp /media/dean/datastore1/datasets/kache_ai/frames/kache_set/scalabel_files/bdd100k/annotations/bdd100k_altered_annotations.json /media/dean/datastore1/data-pipeline-master/data_processing/data/fixtures/kache_construction_prelabel_v1.json

In [ ]:
# !cp /media/dean/datastore1/datasets/darknet/data/bdd_set/bdd100k/annotations/bdd100k_altered_annotations.json /media/dean/datastore1/data-pipeline-master/data_processing/data/fixtures/bdd100k_train_prelabel.json

In [ ]:
# !cp /media/dean/datastore1/datasets/darknet/data/bdd_val_set/bdd100k/annotations/bdd100k_altered_annotations.json /media/dean/datastore1/data-pipeline-master/data_processing/data/fixtures/bdd100k_val_prelabel.json

In [ ]:
# !cp /media/dean/datastore1/datasets/darknet/data/coco_set/bdd100k/annotations/bdd100k_altered_annotations.json /media/dean/datastore1/data-pipeline-master/data_processing/data/fixtures/coco_train2014_prelabel.json

In [ ]:
# !cp /media/dean/datastore1/datasets/darknet/data/coco_val_set/bdd100k/annotations/bdd100k_altered_annotations.json /media/dean/datastore1/data-pipeline-master/data_processing/data/fixtures/coco_val2014_prelabel.json

In [ ]:
# !cd /media/dean/datastore1/data-pipeline-master/ && sudo docker-compose up import_json